In [1]:
#%pip install plotly
#%pip install py2neo

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np 
import pandas as pd
from py2neo import Graph
import matplotlib.pyplot as plt
import time
import plotly.express as px

In [4]:
df = pd.read_csv('C:/Users/Kumar/OneDrive/Documents/Neo4j/CA_1_Excercise_2/code/orders.csv')
df.head()

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,...,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At
0,PT207241,NaN,paid,2022-04-19 23:59:18 +0530,fulfilled,2022-04-20 15:00:35 +0530,no,INR,939.0,0.0,...,NaN,NaN,9.180000e+11,NaN,NaN,Uttar Pradesh,Uttar Pradesh,NaN,NaN,NaN
1,PT207240,NaN,paid,2022-04-19 23:48:24 +0530,fulfilled,2022-04-20 14:01:40 +0530,no,INR,429.0,0.0,...,NaN,NaN,9.160000e+11,NaN,NaN,Telangana,Telangana,NaN,NaN,NaN
2,PT207239,NaN,paid,2022-04-19 23:32:22 +0530,fulfilled,2022-04-20 14:00:48 +0530,no,INR,319.0,0.0,...,NaN,NaN,9.190000e+11,NaN,NaN,Karnataka,Karnataka,NaN,NaN,NaN
3,PT207238,NaN,paid,2022-04-19 23:28:26 +0530,fulfilled,2022-04-20 13:01:06 +0530,no,INR,319.0,0.0,...,NaN,NaN,9.190000e+11,NaN,NaN,Telangana,Telangana,NaN,NaN,NaN
4,PT207237,NaN,paid,2022-04-19 23:27:09 +0530,fulfilled,2022-04-20 14:01:17 +0530,no,INR,1099.0,0.0,...,NaN,NaN,9.200000e+11,NaN,NaN,Himachal Pradesh,Himachal Pradesh,NaN,NaN,NaN


In [229]:
df.columns

Index(['Name', 'Email', 'Financial Status', 'Paid at', 'Fulfillment Status',
       'Fulfilled at', 'Accepts Marketing', 'Currency', 'Subtotal', 'Shipping',
       'Taxes', 'Total', 'Discount Code', 'Discount Amount', 'Shipping Method',
       'Created at', 'Lineitem quantity', 'Lineitem name', 'Lineitem price',
       'Lineitem compare at price', 'Lineitem sku',
       'Lineitem requires shipping', 'Lineitem taxable',
       'Lineitem fulfillment status', 'Billing Name', 'Billing Street',
       'Billing Address1', 'Billing Address2', 'Billing Company',
       'Billing City', 'Billing Zip', 'Billing Province', 'Billing Country',
       'Billing Phone', 'Shipping Name', 'Shipping Street',
       'Shipping Address1', 'Shipping Address2', 'Shipping Company',
       'Shipping City', 'Shipping Zip', 'Shipping Province',
       'Shipping Country', 'Shipping Phone', 'Notes', 'Note Attributes',
       'Cancelled at', 'Payment Method', 'Payment Reference',
       'Refunded Amount', 'Vendor', '

In [230]:
df.drop(['Email', 'Accepts Marketing', 'Currency', 'Subtotal', 'Shipping',
       'Taxes', 'Discount Code', 'Shipping Method', 
       'Created at', 'Lineitem compare at price', 'Lineitem sku',
       'Lineitem requires shipping', 'Lineitem taxable', 'Billing Street',
       'Billing Address1', 'Billing Address2', 'Billing Company',
       'Billing Zip', 'Billing Province', 'Billing Phone', 'Shipping Name', 'Shipping Street',
       'Shipping Address1', 'Shipping Address2', 'Shipping Company',
       'Shipping City', 'Shipping Zip', 'Shipping Province',
       'Shipping Country', 'Shipping Phone', 'Notes', 'Note Attributes',
       'Cancelled at', 'Payment Reference',
       'Refunded Amount', 'Vendor', 'Outstanding Balance', 'Employee',
       'Location', 'Device ID', 'Id', 'Tags', 'Source',
       'Lineitem discount', 'Tax 1 Name', 'Tax 1 Value', 'Tax 2 Name',
       'Tax 2 Value', 'Tax 3 Name', 'Tax 3 Value', 'Tax 4 Name', 'Tax 4 Value',
       'Tax 5 Name', 'Tax 5 Value', 'Phone', 'Receipt Number', 'Duties',
       'Billing Province Name', 'Shipping Province Name', 'Payment ID',
       'Payment Terms Name', 'Next Payment Due At', 'Billing Country', 'Fulfilled at', 'Subtotal','Fulfillment Status'
        ], axis=1, inplace=True)

In [231]:
df.rename(columns = {'Name':'OrderId', 'Billing Name':'CustomerName',
                     'Lineitem name':'ProductName', 'Lineitem price':'Price', 'Paid at':'Date'
                    ,'Lineitem quantity':'Quantity', 'Lineitem fulfillment status':'FulfillmentStatus',
                    'Billing City':'City', 'Financial Status':'FinancialStatus', 'Discount Amount':'DiscountAmount',
                     'Payment Method':'PaymentMethod', 'Risk Level':'RiskLevel'
                    }, inplace = True)

In [232]:
df

,OrderId,FinancialStatus,Date,Total,DiscountAmount,Quantity,ProductName,Price,FulfillmentStatus,CustomerName,City,PaymentMethod,RiskLevel
0,PT207241,paid,2022-04-19 23:59:18 +0530,939.0,2061.0,1,pTron Fusion 10W Bluetooth 5.0 Wireless Speake...,3000,fulfilled,Anil Kumar Upadhyay,Lucknow,CRED,Low
1,PT207240,paid,2022-04-19 23:48:24 +0530,429.0,811.0,1,pTron Bassfest Plus Magnetic in-Ear Bluetooth ...,1240,fulfilled,Mukesh Muddhineni,Hyderabad,CRED,Low
2,PT207239,paid,2022-04-19 23:32:22 +0530,319.0,1581.0,1,"pTron Boom Ultima V2 Dual Driver, in-Ear Gamin...",1900,fulfilled,Prashant Pandey,Bangalore,CRED,Low
3,PT207238,paid,2022-04-19 23:28:26 +0530,319.0,1581.0,1,"pTron Boom Ultima V2 Dual Driver, in-Ear Gamin...",1900,fulfilled,shivaraj medipally,Warangal,CRED,Low
4,PT207237,paid,2022-04-19 23:27:09 +0530,1099.0,3500.0,1,pTron Basspods 992 Active Noise Cancelling (AN...,4599,fulfilled,Pankaj Parihar,Shimla,CRED,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,PT207047,paid,2022-04-18 17:49:41 +0530,229.0,670.0,1,PTron Pride Evo Wired In-Ear Earphone With Mic...,899,fulfilled,Sai Manoj Kumar,Krishna,CRED,Low
196,PT207046,paid,2022-04-18 17:47:57 +0530,899.0,2101.0,1,pTron Fusion 10W Bluetooth 5.0 Wireless Speake...,3000,fulfilled,Vijay Natarajan,Kanchipuram,CRED,Low
197,PT207045,paid,2022-04-18 17:46:23 +0530,399.0,841.0,1,pTron Bassfest Plus Magnetic In-Ear Bluetooth ...,1240,fulfilled,Gurudev Maurya,Jaunpur,CRED,Low
198,PT207044,paid,2022-04-18 17:43:58 +0530,899.0,2101.0,1,pTron Fusion 10W Bluetooth 5.0 Wireless Speake...,3000,fulfilled,Jagdish Gupta,Ahmedabad,CRED,Low


In [233]:
df['Date'] = pd.to_datetime(df['Date'])
df

,OrderId,FinancialStatus,Date,Total,DiscountAmount,Quantity,ProductName,Price,FulfillmentStatus,CustomerName,City,PaymentMethod,RiskLevel
0,PT207241,paid,2022-04-19 23:59:18+05:30,939.0,2061.0,1,pTron Fusion 10W Bluetooth 5.0 Wireless Speake...,3000,fulfilled,Anil Kumar Upadhyay,Lucknow,CRED,Low
1,PT207240,paid,2022-04-19 23:48:24+05:30,429.0,811.0,1,pTron Bassfest Plus Magnetic in-Ear Bluetooth ...,1240,fulfilled,Mukesh Muddhineni,Hyderabad,CRED,Low
2,PT207239,paid,2022-04-19 23:32:22+05:30,319.0,1581.0,1,"pTron Boom Ultima V2 Dual Driver, in-Ear Gamin...",1900,fulfilled,Prashant Pandey,Bangalore,CRED,Low
3,PT207238,paid,2022-04-19 23:28:26+05:30,319.0,1581.0,1,"pTron Boom Ultima V2 Dual Driver, in-Ear Gamin...",1900,fulfilled,shivaraj medipally,Warangal,CRED,Low
4,PT207237,paid,2022-04-19 23:27:09+05:30,1099.0,3500.0,1,pTron Basspods 992 Active Noise Cancelling (AN...,4599,fulfilled,Pankaj Parihar,Shimla,CRED,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,PT207047,paid,2022-04-18 17:49:41+05:30,229.0,670.0,1,PTron Pride Evo Wired In-Ear Earphone With Mic...,899,fulfilled,Sai Manoj Kumar,Krishna,CRED,Low
196,PT207046,paid,2022-04-18 17:47:57+05:30,899.0,2101.0,1,pTron Fusion 10W Bluetooth 5.0 Wireless Speake...,3000,fulfilled,Vijay Natarajan,Kanchipuram,CRED,Low
197,PT207045,paid,2022-04-18 17:46:23+05:30,399.0,841.0,1,pTron Bassfest Plus Magnetic In-Ear Bluetooth ...,1240,fulfilled,Gurudev Maurya,Jaunpur,CRED,Low
198,PT207044,paid,2022-04-18 17:43:58+05:30,899.0,2101.0,1,pTron Fusion 10W Bluetooth 5.0 Wireless Speake...,3000,fulfilled,Jagdish Gupta,Ahmedabad,CRED,Low


In [234]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype                                
---  ------             --------------  -----                                
 0   OrderId            200 non-null    object                               
 1   FinancialStatus    199 non-null    object                               
 2   Date               199 non-null    datetime64[ns, pytz.FixedOffset(330)]
 3   Total              199 non-null    float64                              
 4   DiscountAmount     199 non-null    float64                              
 5   Quantity           200 non-null    int64                                
 6   ProductName        200 non-null    object                               
 7   Price              200 non-null    int64                                
 8   FulfillmentStatus  200 non-null    object                               
 9   CustomerName       199 non-null 

In [235]:
df.isnull().sum()

OrderId              0
FinancialStatus      1
Date                 1
Total                1
DiscountAmount       1
Quantity             0
ProductName          0
Price                0
FulfillmentStatus    0
CustomerName         1
City                 1
PaymentMethod        1
RiskLevel            1
dtype: int64

In [236]:
df = df.dropna()

In [237]:
df.isnull().sum()

OrderId              0
FinancialStatus      0
Date                 0
Total                0
DiscountAmount       0
Quantity             0
ProductName          0
Price                0
FulfillmentStatus    0
CustomerName         0
City                 0
PaymentMethod        0
RiskLevel            0
dtype: int64

In [238]:
df

,OrderId,FinancialStatus,Date,Total,DiscountAmount,Quantity,ProductName,Price,FulfillmentStatus,CustomerName,City,PaymentMethod,RiskLevel
0,PT207241,paid,2022-04-19 23:59:18+05:30,939.0,2061.0,1,pTron Fusion 10W Bluetooth 5.0 Wireless Speake...,3000,fulfilled,Anil Kumar Upadhyay,Lucknow,CRED,Low
1,PT207240,paid,2022-04-19 23:48:24+05:30,429.0,811.0,1,pTron Bassfest Plus Magnetic in-Ear Bluetooth ...,1240,fulfilled,Mukesh Muddhineni,Hyderabad,CRED,Low
2,PT207239,paid,2022-04-19 23:32:22+05:30,319.0,1581.0,1,"pTron Boom Ultima V2 Dual Driver, in-Ear Gamin...",1900,fulfilled,Prashant Pandey,Bangalore,CRED,Low
3,PT207238,paid,2022-04-19 23:28:26+05:30,319.0,1581.0,1,"pTron Boom Ultima V2 Dual Driver, in-Ear Gamin...",1900,fulfilled,shivaraj medipally,Warangal,CRED,Low
4,PT207237,paid,2022-04-19 23:27:09+05:30,1099.0,3500.0,1,pTron Basspods 992 Active Noise Cancelling (AN...,4599,fulfilled,Pankaj Parihar,Shimla,CRED,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,PT207047,paid,2022-04-18 17:49:41+05:30,229.0,670.0,1,PTron Pride Evo Wired In-Ear Earphone With Mic...,899,fulfilled,Sai Manoj Kumar,Krishna,CRED,Low
196,PT207046,paid,2022-04-18 17:47:57+05:30,899.0,2101.0,1,pTron Fusion 10W Bluetooth 5.0 Wireless Speake...,3000,fulfilled,Vijay Natarajan,Kanchipuram,CRED,Low
197,PT207045,paid,2022-04-18 17:46:23+05:30,399.0,841.0,1,pTron Bassfest Plus Magnetic In-Ear Bluetooth ...,1240,fulfilled,Gurudev Maurya,Jaunpur,CRED,Low
198,PT207044,paid,2022-04-18 17:43:58+05:30,899.0,2101.0,1,pTron Fusion 10W Bluetooth 5.0 Wireless Speake...,3000,fulfilled,Jagdish Gupta,Ahmedabad,CRED,Low


In [239]:
df.to_csv('processed_orders.csv')

In [240]:
df.to_parquet('preprocessed_ecommerce.parquet')

In [380]:
ip="44.203.203.126"
port="7474"
pwd="jackets-housings-compromises"
graph = Graph("bolt://44.203.203.126:7687", auth=("neo4j", pwd))

In [381]:
UPLOAD_DATA = True

def run_request(request,UPLOAD_DATA):
    if (UPLOAD_DATA):
        start_time = time.time()
        print(graph.run(request)) 
        exec_time = time.time() - start_time
        print(f'-- {exec_time} seconds --')

In [385]:
#request_delete ="""
#MATCH (n) DETACH DELETE n;"""
#run_request(request_delete,UPLOAD_DATA)

(No data)
-- 2.2982046604156494 seconds --


In [386]:
df.keys()

Index(['OrderId', 'FinancialStatus', 'Date', 'Total', 'DiscountAmount',
       'Quantity', 'ProductName', 'Price', 'FulfillmentStatus', 'CustomerName',
       'City', 'PaymentMethod', 'RiskLevel'],
      dtype='object')

In [388]:
request ='''
CREATE (o:Order {id:$OrderId ,product: $ProductName})
SET 
o.FinancialStatus = $FinancialStatus,
o.Date = $Date, 
o.Total = $Total,
o.DiscountAmount=$DiscountAmount,
o.OrderId=$OrderId,
o.Quantity = $Quantity,
o.ProductName = $ProductName,
o.Price = $Price, 
o.FulfillmentStatus = $FulfillmentStatus,
o.CustomerName=$CustomerName,
o.City = $City,
o.PaymentMethod=$PaymentMethod,
o.RiskLevel = $RiskLevel
'''

start_time = time.time()
count=df.shape[0]
if (UPLOAD_DATA):
    for OrderId, row in df.iterrows():
        graph.run(request, OrderId=row['OrderId'],
                  FinancialStatus=None if pd.isnull(row['FinancialStatus']) else row['FinancialStatus'],
                  Date= None if pd.isnull(row['Date']) else row['Date'],
                  Total=None if pd.isnull(row['Total']) else row['Total'],          
                  DiscountAmount= None if pd.isnull(row['DiscountAmount']) else row['DiscountAmount'],
                  Quantity=None if pd.isnull(row['Quantity']) else row['Quantity'],
                  ProductName= None if pd.isnull(row['ProductName']) else row['ProductName'],
                  Price= None if pd.isnull(row['Price']) else row['Price'],
                  FulfillmentStatus=None if pd.isnull(row['FulfillmentStatus']) else row['FulfillmentStatus'],          
                  CustomerName= None if pd.isnull(row['CustomerName']) else row['CustomerName'],
                  City=None if pd.isnull(row['City']) else row['City'],
                  PaymentMethod= None if pd.isnull(row['PaymentMethod']) else row['PaymentMethod'],
                  RiskLevel=None if pd.isnull(row['RiskLevel']) else row['RiskLevel'])
        
    if (OrderId%100==0):
        exec_time = time.time()- start_time
print(f'{OrderId}/{df.shape[0]} done ... -- {exec_time} seconds --')
exec_time = time.time()- start_time
print(f' ... End ... -- {exec_time} seconds --')

199/199 done ... -- 23.62553381919861 seconds --
 ... End ... -- 23.34676742553711 seconds --


### Basic graph exploratory data analysis (EDA)

In [389]:
# customer to date
request ="""
MATCH (o:Order)
WITH o
MERGE (d:Date{od:o.Date})
MERGE (c:Customer {name: trim(o.CustomerName)})
MERGE (c)-[r:PURCHASED_ON]->(d);
"""    
run_request(request,UPLOAD_DATA)

(No data)
-- 0.9028902053833008 seconds --


In [391]:
# customer to product name
request ="""
MATCH (o:Order)
WITH o
MERGE (p:Product{pn:o.ProductName})
MERGE (c:Customer {name: trim(o.CustomerName)})
MERGE (c)-[r:PURCHASED]->(p);
"""    
run_request(request,UPLOAD_DATA)

(No data)
-- 0.6206815242767334 seconds --


In [390]:
# product name to order id
request ="""
MATCH (o:Order)
WHERE o.OrderId IS NOT NULL
WITH o
MERGE (oid:OrderId{oi:o.OrderId})
MERGE (p:Product{pn:o.ProductName})
MERGE (p)-[r:ID]->(oid);
""" 
run_request(request,UPLOAD_DATA)

(No data)
-- 0.5042617321014404 seconds --


In [392]:
# product name to quantity
request ="""
MATCH (o:Order)
WHERE o.OrderId IS NOT NULL
WITH o
MERGE (q:ItemQuantity{qn:o.Quantity})
MERGE (p:Product{pn:o.ProductName})
MERGE (p)-[r:PRODUCT_QUANTITY]->(q);
"""    
run_request(request,UPLOAD_DATA)

(No data)
-- 0.38634443283081055 seconds --


In [393]:
# orderid to quantity
request ="""
MATCH (o:Order)
WHERE o.OrderId IS NOT NULL
WITH o
MERGE (q:ItemQuantity{qn:o.Quantity})
MERGE (oid:OrderId{oi:o.OrderId})
MERGE (oid)-[r:RELATED_TO]->(q);
"""    
run_request(request,UPLOAD_DATA)

(No data)
-- 0.49872326850891113 seconds --


In [394]:
# product name to price
request ="""
MATCH (o:Order)
WITH o
MERGE (pr:Price{name:o.Price})
MERGE (p:Product{pn:o.ProductName})
MERGE (p)-[r:PRODUCT_PRICE]->(pr);
"""    
run_request(request,UPLOAD_DATA)

(No data)
-- 0.5456371307373047 seconds --


In [395]:
# price to discount amount
request ="""
MATCH (o:Order)
WITH o
MERGE (disc:Discount{da:o.DiscountAmount})
MERGE (pr:Price{name:o.Price})
MERGE (pr)-[r:HAS_DISCOUNT]->(disc);
"""    
run_request(request,UPLOAD_DATA)

(No data)
-- 0.3649613857269287 seconds --


In [396]:
# discount amount to total
request ="""
MATCH (o:Order)
WITH o
MERGE (t:Total{to:o.Total})
MERGE (disc:Discount{da:o.DiscountAmount})
MERGE (disc)-[r:AFTER_DISCOUNT]->(t);
"""    
run_request(request,UPLOAD_DATA)

(No data)
-- 0.375136137008667 seconds --


In [397]:
# total to payment method
request ="""
MATCH (o:Order)
WITH o
MERGE (pm:PaymentMethod{pmt:o.PaymentMethod})
MERGE (t:Total{to:o.Total})
MERGE (t)-[r:PAYMENT_TYPE]->(pm);
"""    
run_request(request,UPLOAD_DATA)

(No data)
-- 0.32384753227233887 seconds --


In [398]:
# payment method to city
request ="""
MATCH (o:Order)
WITH o
MERGE (c:City{ct:o.City})
MERGE (pm:PaymentMethod{pmt:o.PaymentMethod})
MERGE (pm)-[r:DELIVERS_TO]->(c);
"""    
run_request(request,UPLOAD_DATA)

(No data)
-- 0.4098992347717285 seconds --


In [399]:
# city to fulfilment status
request ="""
MATCH (o:Order)
WITH o
MERGE (f:FulfillmentStatus{fs:o.FulfillmentStatus})
MERGE (c:City{ct:o.City})
MERGE (c)-[r:DELIVERED_STATUS]->(f);
"""    
run_request(request,UPLOAD_DATA)

(No data)
-- 0.451526403427124 seconds --


In [400]:
# Data visualization
graph.run("CALL db.schema.visualization()").data()

[{'nodes': [Node('Order', constraints=[], indexes=[], name='Order'),
   Node('_Bloom_Perspective_', constraints=["Constraint( id=4, name='constraint_b8a3620a', type='UNIQUENESS', schema=(:_Bloom_Perspective_ {id}), ownedIndex=3 )"], indexes=[], name='_Bloom_Perspective_'),
   Node('Discount', constraints=[], indexes=[], name='Discount'),
   Node('Customer', constraints=[], indexes=[], name='Customer'),
   Node('Product', constraints=[], indexes=[], name='Product'),
   Node('City', constraints=[], indexes=[], name='City'),
   Node('OrderId', constraints=[], indexes=[], name='OrderId'),
   Node('Date', constraints=[], indexes=[], name='Date'),
   Node('FulfillmentStatus', constraints=[], indexes=[], name='FulfillmentStatus'),
   Node('Price', constraints=[], indexes=[], name='Price'),
   Node('ItemQuantity', constraints=[], indexes=[], name='ItemQuantity'),
   Node('Total', constraints=[], indexes=[], name='Total'),
   Node('PaymentMethod', constraints=[], indexes=[], name='PaymentMethod

In [401]:
result = {"label": [], "count": []}
for label in graph.run("CALL db.labels()").to_series():
    query = f"MATCH (:`{label}`) RETURN count(*) as count"
    count = graph.run(query).to_data_frame().iloc[0]['count']
    result["label"].append(label)
    result["count"].append(count)
nodes_df = pd.DataFrame(data=result)
nodes_df.sort_values("count",ascending=False)

,label,count
0,Order,398
2,Date,199
4,OrderId,199
1,Customer,189
10,City,91
3,Product,29
7,Discount,26
8,Total,23
6,Price,17
11,FulfillmentStatus,3


In [402]:
fig=px.bar(nodes_df,x='label',y='count',title="Node Cardinalities")
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [403]:
result = {"RelationshipType": [], "count": []}
for relationship_type in graph.run("CALL db.relationshipTypes()").to_series():
    query = f"MATCH ()-[:`{relationship_type}`]->() RETURN count(*) as count"
    count = graph.run(query).to_data_frame().iloc[0]['count']
    result["RelationshipType"].append(relationship_type)
    result["count"].append(count)
rels_df = pd.DataFrame(data=result)
rels_df.sort_values("count",ascending=False)

,RelationshipType,count
0,PURCHASED_ON,199
8,ID,199
10,RELATED_TO,199
1,PURCHASED,197
7,DELIVERED_STATUS,95
6,DELIVERS_TO,94
4,AFTER_DISCOUNT,32
3,HAS_DISCOUNT,31
2,PRODUCT_PRICE,29
9,PRODUCT_QUANTITY,29


In [404]:
fig=px.bar(rels_df,x='RelationshipType',y='count',title="Relationship Cardinalities")
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [405]:
analyze_top_10_customers="""
MATCH (c:Customer)-[r:PURCHASED]->(p:Product)
WITH c,collect(p.pn) as products,count(*) as total
RETURN  c.name as Customer, products As PurchasedProducts,total As TotalProducts
ORDER BY total DESC
LIMIT 25
"""
graph.run(analyze_top_10_customers).to_data_frame().head(10)

,Customer,PurchasedProducts,TotalProducts
0,Shreyas M R,[PTron Tangent Plus V2 Wireless Bluetooth In-E...,6
1,Nagnath Bhusnar,[pTron Bassfest Plus Magnetic in-Ear Bluetooth...,2
2,Anjali Chopra,[pTron Basspods 992 Active Noise Cancelling (A...,2
3,Ramani Nair,[pTron Fusion 10W Bluetooth 5.0 Wireless Speak...,2
4,Rajiv Kumar,[pTron Fusion 10W Bluetooth 5.0 Wireless Speak...,1
5,Suraj Duftuwar,[pTron Fusion 10W Bluetooth 5.0 Wireless Speak...,1
6,Vaidyanathan S,[pTron Fusion 10W Bluetooth 5.0 Wireless Speak...,1
7,Sarath G S,[pTron Fusion 10W Bluetooth 5.0 Wireless Speak...,1
8,Varnipa Bhendigeri,[pTron Fusion 10W Bluetooth 5.0 Wireless Speak...,1
9,Alkachenu Veerabadrudu,[pTron Fusion 10W Bluetooth 5.0 Wireless Speak...,1


### Algorithm Tests

#### Degree Centrality

In [406]:
request = '''
match (n:Product)-[r]->()
with n, count(r) as c
return n,c
order by c desc
'''
run_request(request,UPLOAD_DATA)

 n                                                                                                                                                                                                                            |  c 
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----
 (_804:Product {pn: 'pTron Bassbuds Tango ENC (Environmental Noise Cancellation), Dedicated Movie Mode, 40Hrs Playtime, Bluetooth 5.1 Wireless Headphones, Deep Bass, Touch Control TWS & Type-C Fast Charging (Black)'})     | 35 
 (_797:Product {pn: 'pTron Boom Ultima V2 Dual Driver, in-Ear Gaming Wired Headphones with in-line Mic, Volume Control, Passive Noise Cancelling Boom 3 Earphones with 3.5mm Audio Jack & 1.2M Tangle-Free Cable - (Black)'}) | 28 
 (_793:Product {pn: 'pTron Fusion 10W Bluetooth 5.0 Wireless Speaker, Stereo Sound Audio

In [407]:
request = '''
MATCH (c:Customer)-[r:PURCHASED]->(p:Product)
RETURN c.name as customer, collect(p.pn) as products, count(r) as outgoingDegree
ORDER BY outgoingDegree DESC
'''
run_request(request,UPLOAD_DATA)

 customer        | products                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | outgoingDegree 
-----------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [408]:
request = '''
MATCH (c:Customer)
MATCH (p:Product)
WHERE NOT EXISTS ( (c)-[:PURCHASED]->(p) )
WITH c.name as user, p.pn as product, gds.alpha.linkprediction.commonNeighbors(c, p, {
    relationshipQuery: "PURCHASED",
    direction: "BOTH"
}) as score
RETURN user, product, score
ORDER BY score DESC
'''
run_request(request,UPLOAD_DATA)

 user               | product                                                                                                                                                                           | score 
--------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------
 Mukesh Muddhineni  | pTron Fusion 10W Bluetooth 5.0 Wireless Speaker, Stereo Sound Audio, 10Hrs Music Playback, Speaker with Mic, Portable Indoor Outdoor Speaker with Aux/TF Card/USB Ports - (Black) |   0.0 
 Prashant Pandey    | pTron Fusion 10W Bluetooth 5.0 Wireless Speaker, Stereo Sound Audio, 10Hrs Music Playback, Speaker with Mic, Portable Indoor Outdoor Speaker with Aux/TF Card/USB Ports - (Black) |   0.0 
 shivaraj medipally | pTron Fusion 10W Bluetooth 5.0 Wireless Speaker, Stereo Sound Audio, 10Hrs Music Playback, Speaker with Mic, Portable Indoor Outdoor Speaker w

In [409]:
request = '''
MATCH (c:Customer)
MATCH (p:Product)
WHERE NOT EXISTS ( (c)-[:PURCHASED]->(p) )
WITH c.name as user, p.pn as product, gds.alpha.linkprediction.totalNeighbors (c, p, {
    relationshipQuery: "PURCHASED",
    direction: "BOTH"
}) as score
RETURN user, product, score
ORDER BY score DESC
'''
run_request(request,UPLOAD_DATA)

 user          | product                                                                                                                                                                                           | score 
---------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------
 Shreyas M R   | pTron Bassbuds Tango ENC (Environmental Noise Cancellation), Dedicated Movie Mode, 40Hrs Playtime, Bluetooth 5.1 Wireless Headphones, Deep Bass, Touch Control TWS & Type-C Fast Charging (Black) |  39.0 
 Ramani Nair   | pTron Bassbuds Tango ENC (Environmental Noise Cancellation), Dedicated Movie Mode, 40Hrs Playtime, Bluetooth 5.1 Wireless Headphones, Deep Bass, Touch Control TWS & Type-C Fast Charging (Black) |  35.0 
 Anjali Chopra | pTron Bassbuds Tango ENC (Environmental Noise Cancellation), Dedicated Movie Mode, 40Hrs Playtime, Blue